In [95]:
import findspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import desc, col, countDistinct, rank
from pyspark.sql.window import Window

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

Adjunto como recurso a esta lección están los archivos movies.csv y movie_ratings.csv. En ambos archivos las columnas están delimitadas por un pip(“|”).

Cada línea del archivo movies.csv representa a un actor que actuó en una película. Si una película tiene diez actores, habrá diez filas para esa película en particular.

In [6]:
df_movies = spark.read.option('header', 'true').option('delimiter', '|').csv('./data/movies/movies.csv')

In [11]:
df_movie_ratings = spark.read.option('header', 'true').option('delimiter', '|').csv('./data/movies/movie_ratings.csv')

In [9]:
df_movies.printSchema()

root
 |-- actor: string (nullable = true)
 |-- pelicula: string (nullable = true)
 |-- año: string (nullable = true)



In [10]:
df_movies.show(5)

+-----------------+-------------+----+
|            actor|     pelicula| año|
+-----------------+-------------+----+
|McClure, Marc (I)|Freaky Friday|2003|
|McClure, Marc (I)| Coach Carter|2005|
|McClure, Marc (I)|  Superman II|1980|
|McClure, Marc (I)|    Apollo 13|1995|
|McClure, Marc (I)|     Superman|1978|
+-----------------+-------------+----+
only showing top 5 rows



In [12]:
df_movie_ratings.printSchema()

root
 |-- valoracion: string (nullable = true)
 |-- pelicula: string (nullable = true)
 |-- año: string (nullable = true)



In [14]:
df_movie_ratings.show(5)

+----------+--------------------+----+
|valoracion|            pelicula| año|
+----------+--------------------+----+
|    1.6339|'Crocodile' Dunde...|1988|
|    7.6177|                  10|1979|
|    1.2864|10 Things I Hate ...|1999|
|    0.3243|           10,000 BC|2008|
|    0.3376|      101 Dalmatians|1996|
+----------+--------------------+----+
only showing top 5 rows



Calcule la cantidad de películas en las que participó cada actor. La salida debe tener dos columnas: actor y conteo. La salida debe ordenarse por el conteo en orden descendente.

In [20]:
df_pelisXactor = df_movies.drop('año').groupBy('actor').agg(count('pelicula').alias('pelis_x_actor')).orderBy(desc('pelis_x_actor')).show(5)

+------------------+-------------+
|             actor|pelis_x_actor|
+------------------+-------------+
|  Tatasciore, Fred|           38|
|     Welker, Frank|           38|
|Jackson, Samuel L.|           32|
|     Harnell, Jess|           31|
|     Willis, Bruce|           27|
+------------------+-------------+
only showing top 5 rows



In [72]:
# Otro modo más fácil de resolverlo:

df_movies.groupBy('actor').count().orderBy(desc('count')).show(5)

+------------------+-----+
|             actor|count|
+------------------+-----+
|  Tatasciore, Fred|   38|
|     Welker, Frank|   38|
|Jackson, Samuel L.|   32|
|     Harnell, Jess|   31|
|     Willis, Bruce|   27|
+------------------+-----+
only showing top 5 rows



Calcule la cantidad de películas producidas cada año. La salida debe tener tres columnas: año, siglo al que pertenece el año y conteo. La salida debe ordenarse por el conteo en orden descendente.

In [26]:
df_movies.agg(countDistinct('pelicula')).collect()

[Row(count(pelicula)=1409)]

In [28]:
df_movie_ratings.agg(countDistinct('pelicula')).collect()

[Row(count(pelicula)=3536)]

In [38]:
# Comprobamos que no hay duplicados en df_movie_ratings

duplicate_rows = df_movie_ratings.groupBy(df_movie_ratings.columns).count().where(col("count") > 1)
duplicate_rows.show()

+----------+--------+---+-----+
|valoracion|pelicula|año|count|
+----------+--------+---+-----+
+----------+--------+---+-----+



In [70]:
# Usaremos df_movie_ratings porque contiene más películas

df_pelisXyear = df_movie_ratings.groupBy('año').agg(count('pelicula').alias('pelis_x_año')).orderBy('año')
df_pelisXyear.show(5)

+----+-----------+
| año|pelis_x_año|
+----+-----------+
|1937|          1|
|1939|          2|
|1940|          2|
|1942|          1|
|1946|          1|
+----+-----------+
only showing top 5 rows



In [116]:
(df_pelisXyear.withColumn("año",col("año").cast("integer")) # Pasamos la columna año de string a entero
              .withColumn("siglo", ((col('año')/100)+1).cast("integer")) # Creamos la columna siglo
              .select(col('año'), col('siglo'), col('pelis_x_año')) # Hacemos un select para obtener las columnas en el orden deseado
              .show(5)
)

+----+-----+-----------+
| año|siglo|pelis_x_año|
+----+-----+-----------+
|1937|   20|          1|
|1939|   20|          2|
|1940|   20|          2|
|1942|   20|          1|
|1946|   20|          1|
+----+-----+-----------+
only showing top 5 rows



Obtenga la película con la calificación más alta por año. La salida debe tener solo una película por año y debe contener tres columnas: año, título de la película y valoración.

In [93]:
df_movie_ratings.show(5)

+----------+--------------------+----+
|valoracion|            pelicula| año|
+----------+--------------------+----+
|    1.6339|'Crocodile' Dunde...|1988|
|    7.6177|                  10|1979|
|    1.2864|10 Things I Hate ...|1999|
|    0.3243|           10,000 BC|2008|
|    0.3376|      101 Dalmatians|1996|
+----------+--------------------+----+
only showing top 5 rows



In [106]:
windowSpec = Window.partitionBy('año').orderBy(desc('valoracion'))

In [119]:
(df_movie_ratings
                .withColumn('rank', rank().over(windowSpec)) # Aplicamos rank a la partición de ventana
                .filter(col('rank')==1) # Filtramos los que estén en el primer puesto
                .select(col('año'), col('pelicula'), col('valoracion').alias('maxima_valoracion')) # Seleccionamos las columnas que nos interesan
                .show(truncate=False) # Imprimimos el resultado
)

+----+-------------------------------+-----------------+
|año |pelicula                       |maxima_valoracion|
+----+-------------------------------+-----------------+
|1937|Snow White and the Seven Dwarfs|2.2207           |
|1939|The Wizard of Oz               |7.9215           |
|1940|Pinocchio                      |7.8557           |
|1942|Bambi                          |1.5053           |
|1946|Song of the South              |7.602            |
|1950|Cinderella                     |9.4226           |
|1953|Peter Pan                      |5.4756           |
|1954|Rear Window                    |10.7625          |
|1955|Lady and the Tramp             |5.1258           |
|1956|The Ten Commandments           |7.3377           |
|1959|Sleeping Beauty                |6.3919           |
|1960|Spartacus                      |8.8223           |
|1961|One Hundred and One Dalmatians |0.6726           |
|1962|Dr. No                         |6.6427           |
|1963|It's a Mad Mad Mad Mad Wo